In [34]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm 

class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.classes = os.listdir(data_dir)
        print(self.classes)  # training directories
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []

        for class_dir in self.classes:
            class_path = os.path.join(data_dir, class_dir)
            if os.path.isdir(class_path):
                image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
                self.images.extend([(os.path.join(class_path, f), self.class_to_idx[class_dir]) for f in image_files])

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize to a fixed size
            transforms.ToTensor(),  # Convert to PyTorch tensor
            transforms.Normalize(mean=[0.485], std=[0.229])  # Normalize 
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path, label = self.images[idx]
        try:
            with open(image_path, 'rb') as file:
                image = Image.open(file)
                if image.mode != 'L':
                    image = image.convert('L')  # Convert to grayscale
                if self.transform:
                    tensor_image = self.transform(image)
                else:
                    tensor_image = image
                return tensor_image, label
        except Exception as e:
            
            raise ValueError(f'Path or data is wrong {e}')

train_path = "train" # Update directory path - preferrably put the code with the data
val_path = "val"


# instance of custom dataset
train_dataset = MyDataset(train_path)
val_dataset = MyDataset(val_path)

# parameters for DataLoader
batch_size = 64
shuffle = True
num_workers = 0  # Adjust this based on your system's capabilities, 4 if system is good

# DataLoader instance
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

#for batch_idx in train_loader:
    #print(batch_idx)
    
        
    # Process your image data batch here
#    pass

['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']


In [32]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import optuna
from optuna.trial import TrialState



#Define some activation functions
activation_functions = {
    'relu': F.relu,
    'sigmoid': torch.sigmoid,
    'tanh': torch.tanh,
    'leaky_relu': F.leaky_relu,
    'elu': F.elu,
}

class Net(nn.Module):
    def __init__(self, trial):
        super(Net, self).__init__()
        in_size = 256

            # Define the model parameters and other hyperparameters
        #Number of layers
        num_conv_layers = trial.suggest_int('num_conv_layers', 2, 4)
        #Number of filters
        num_filters = [trial.suggest_int('num_filters_{}'.format(i), 16, 128) for i in range(num_conv_layers)]
        #Number of neurons
        num_neurons = trial.suggest_int('num_neurons', 100, 500)
        #Dropout of conv2 (if applicable)
        drop_conv2 = trial.suggest_uniform('drop_conv2', 0.0, 0.5)
        #Dropout of the first connected layer
        drop_fc1 = trial.suggest_uniform('drop_fc1', 0.0, 0.5)

        # Suggest activation functions (used in forward pass, here a name is given that will be used in the dict above)
        self.conv_activation_name = trial.suggest_categorical('conv_activation', list(activation_functions.keys()))
        self.fc_activation_name = trial.suggest_categorical('fc_activation', list(activation_functions.keys()))

        self.convs = nn.ModuleList([nn.Conv2d(1, num_filters[0], kernel_size=(3, 3))])
        out_size = in_size - 2  # 3 - 1 for kernel size minus 1
        out_size = out_size // 2

        for i in range(1, num_conv_layers):
            self.convs.append(nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=(3, 3)))
            out_size = (out_size - 2) // 2

        self.conv2_drop = nn.Dropout2d(p=drop_conv2)
        self.out_feature = num_filters[-1] * out_size * out_size
        self.fc1 = nn.Linear(self.out_feature, num_neurons)
        self.fc2 = nn.Linear(num_neurons, 1)
        self.p1 = drop_fc1

        # Initialize weights
        for conv in self.convs:
            nn.init.kaiming_normal_(conv.weight, nonlinearity='relu')
            if conv.bias is not None:
                nn.init.constant_(conv.bias, 0)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')

    def forward(self, x):
        conv_activation = activation_functions[self.conv_activation_name]
        fc_activation = activation_functions[self.fc_activation_name]

        for i, conv in enumerate(self.convs):
            x = conv(x)
            x = conv_activation(x)
            x = F.max_pool2d(x, 2)
            if i == 2:
                x = self.conv2_drop(x)

        x = x.view(-1, self.out_feature)
        x = fc_activation(self.fc1(x))
        x = F.dropout(x, p=self.p1, training=self.training)
        x = self.fc2(x)

        return torch.sigmoid(x)


def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.BCELoss()  # Use Binary Cross Entropy Loss for binary classification
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        target = target.float().unsqueeze(1)  # Ensure target is the correct shape and type
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()  

    
    print('Loss: {:.6f}, Epoch: {}'.format(loss.item(),epoch))                                   


def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.BCELoss()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            target = target.float()
            output = output.view(-1)
            test_loss += criterion(output, target)
            # Binary classification prediction
            pred = (output > 0.5).float()  # Thresholding at 0.5
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

def objective(trial):
    
    #Load data
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    #Choose cuda if available, else cpu. 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

    #Only 10 for hyperparameter tuning
    num_epochs = 10

    #load data
    model = Net(trial).to(device)

        # Generate the optimizers
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)                               
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

        # Training of the model
    for epoch in range(num_epochs):
        train_model(model, device, train_loader, optimizer, epoch)  # Train the model
        accuracy = test_model(model, device, val_loader)   # Evaluate the model

        # For pruning (stops trial early if not promising)
        trial.report(accuracy, epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    trial.set_user_attr('model', model)

    return accuracy

In [33]:
def save_best_model(study, trial):
    if study.best_trial.number == trial.number:
        best_model = trial.user_attrs['model']
        torch.save(best_model, 'best_model_2.pth')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, callbacks=[save_best_model])

print("Best hyperparameters: ", study.best_trial.params)

[I 2024-05-15 21:09:54,210] A new study created in memory with name: no-name-1c4e60de-fb83-40d4-bb57-0c9f6beed22b
C:\Users\masv\AppData\Local\Temp\ipykernel_22316\494596573.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  drop_conv2 = trial.suggest_uniform('drop_conv2', 0.0, 0.5)
C:\Users\masv\AppData\Local\Temp\ipykernel_22316\494596573.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  drop_fc1 = trial.suggest_uniform('drop_fc1', 0.0, 0.5)


Loss: 31.250000, Epoch: 0

Test set: Average loss: 3.1250, Accuracy: 8/16 (50%)



[I 2024-05-15 21:20:28,964] Trial 0 finished with value: 50.0 and parameters: {'num_conv_layers': 2, 'num_filters_0': 98, 'num_filters_1': 111, 'num_neurons': 813, 'drop_conv2': 0.08152126297232581, 'drop_fc1': 0.46066056740821915, 'conv_activation': 'relu', 'fc_activation': 'leaky_relu', 'optimizer': 'RMSprop', 'lr': 0.0009449630558906713}. Best is trial 0 with value: 50.0.
[W 2024-05-15 21:20:43,072] Trial 1 failed with parameters: {'num_conv_layers': 4, 'num_filters_0': 49, 'num_filters_1': 52, 'num_filters_2': 30, 'num_filters_3': 19, 'num_neurons': 860, 'drop_conv2': 0.19181357997200632, 'drop_fc1': 0.237740202698265, 'conv_activation': 'leaky_relu', 'fc_activation': 'sigmoid', 'optimizer': 'RMSprop', 'lr': 0.00034136150110160746} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\masv\AppData\Local\miniforge3\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\User

KeyboardInterrupt: 

In [7]:
model = torch.load('best_model.pth',map_location=torch.device('cpu'))
#model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_model(model,device,val_loader)

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [18]:
model

OrderedDict([('convs.0.weight',
              tensor([[[[ 0.1228,  0.0830, -0.1227],
                        [-0.0631,  0.1482,  0.3272],
                        [-0.0465,  0.2329, -0.4346]],
              
                       [[-0.3679, -0.0526, -0.2006],
                        [-0.4310,  0.4524,  0.0892],
                        [ 0.2695, -0.0429,  0.5000]],
              
                       [[-0.3989,  0.0224, -0.3173],
                        [-0.0816,  0.5339, -0.0816],
                        [-0.5659, -0.1707, -0.1542]]],
              
              
                      [[[-0.0384,  0.0640, -0.4782],
                        [ 0.2421,  0.2640, -0.0802],
                        [ 0.3875,  0.6002,  0.4804]],
              
                       [[-0.0506, -0.2102, -0.3600],
                        [ 0.2909, -0.2469,  0.4300],
                        [ 0.1046,  0.2658,  0.0199]],
              
                       [[-0.2797, -0.3294, -0.3609],
                        

In [12]:
model.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'